In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM_LBLNEW['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
DS_FLUX_WGT.sel(igg=1).sel(g=1).to_dataframe();

In [14]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    
    


In [15]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    s_outputs_byg = 'Outputs by g-interval'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_outputs_byg))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [16]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 1013f91
molecule = {'co2': 0.0004}
band = [4]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a22ab94
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 4]
  nv = 1000
  option_compute_btable = 1
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66, 1.66, 1.66)]
  wgt = [(0, 0.5, 0.4, 0.6, 0.65), (0.7, 0.7, 0.55, 0.6)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`                 
            layer     coolrg                    i layer     coolrg
pressure                                                          
0.000312        1  66.811869                    1     1  44.714254
0.000750        2  30.192711                    1     2  32.076582
0.001052        3  28.255311                    1     3  31.058694
0.001476        4  25.341715                    1     4  28.380811
0.002070        5  22.174221                    1     5  24.342332
0.002904        6  19.226535                    1     6  19.784353
0.004074        7  16.722187                    1     7  15.804571
0.005714        8  14.724929                    1     8  13.239320
0.008015        9  13.199432                    1     9  11.985891
0.011243       10  12.055478                    1    10  11.364303
0.015771       11  11.189547                    1    11  10.773306
0.022122       12  10.518709                    1    12  10.025188
0.031031       13  10.089954                    1    13   9.372094
0.043528       14   9.446098                    1    14   8.711925
0.061057       15   8.347313                    1    15   7.987662
0.085645       16   7.340092                    1    16   7.552884
0.120136       17   6.662366                    1    17   7.298054
0.168516       18   6.248298                    1    18   6.955353
0.236378       19   6.084259                    1    19   6.519815
0.331549       20   6.196347                    1    20   6.213776
0.465100       21   6.809821                    1    21   6.506363
0.652400       22   6.510550                    1    22   6.208329
0.915100       23   5.072440                    1    23   4.971479
1.283650       24   3.910509                    1    24   3.886141
1.800600       25   3.126184                    1    25   3.010487
2.525700       26   2.505188                    1    26   2.337712
3.542800       27   1.954727                    1    27   1.948574
4.969550       28   1.571513                    1    28   1.699260
6.970850       29   1.401774                    1    29   1.507472
9.778100       30   1.262336                    1    30   1.299257
13.715850      31   1.074496                    1    31   1.104575
19.239350      32   0.841936                    1    32   0.829572
26.987250      33   0.698846                    1    33   0.668487
37.855300      34   0.648636                    1    34   0.677182
53.100050      35   0.551658                    1    35   0.600669
73.887500      36   0.434397                    1    36   0.431625
97.662500      37   0.339799                    1    37   0.316659
121.437500     38   0.247667                    1    38   0.250845
145.212500     39   0.170539                    1    39   0.198201
168.987500     40   0.126622                    1    40   0.161723
192.762500     41   0.095783                    1    41   0.125035
216.537500     42   0.070982                    1    42   0.088939
240.312500     43   0.047261                    1    43   0.052509
264.087500     44   0.012151                    1    44   0.006720
287.862500     45  -0.067821                    1    45  -0.081792
311.637500     46  -0.102537                    1    46  -0.120709
335.412500     47  -0.036092                    1    47  -0.054391
359.187500     48   0.019780                    1    48   0.004912
382.962500     49   0.036545                    1    49   0.025602
406.737500     50   0.044235                    1    50   0.036242
430.512500     51   0.048698                    1    51   0.042436
454.287500     52   0.051478                    1    52   0.045171
478.062500     53   0.053335                    1    53   0.048028
501.837500     54   0.054652                    1    54   0.049812
525.612500     55   0.055710                    1    55   0.051708
549.387500     56   0.056489                    1    56   0.053210
573.162500     57   0.056954                    1    57   0.0

# Fluxes by Layer

CRD                       CLIRAD             \
                        flug       fldg      fnetg      i       flug   
pressure    level                                                      
0.000000    1     -14.290363   0.000000 -14.290363      1 -14.145341   
0.000624    2     -14.291621   0.006200 -14.285421      1 -14.146583   
0.000876    3     -14.292224   0.007703 -14.284522      1 -14.147119   
0.001229    4     -14.292966   0.009625 -14.283341      1 -14.147788   
0.001723    5     -14.293870   0.012014 -14.281856      1 -14.148613   
0.002417    6     -14.294965   0.014932 -14.280033      1 -14.149624   
0.003391    7     -14.296284   0.018469 -14.277816      1 -14.150851   
0.004757    8     -14.297864   0.022754 -14.275111      1 -14.152333   
0.006672    9     -14.299741   0.027972 -14.271769      1 -14.154105   
0.009359    10    -14.301953   0.034385 -14.267568      1 -14.156204   
0.013128    11    -14.304530   0.042344 -14.262186      1 -14.158657   
0.018415    12    -14.307484   0.052305 -14.255179      1 -14.161473   
0.025830    13    -14.310774   0.064835 -14.245939      1 -14.164606   
0.036232    14    -14.314111   0.080606 -14.233506      1 -14.167784   
0.050823    15    -14.317192   0.100013 -14.217179      1 -14.170738   
0.071291    16    -14.320175   0.123234 -14.196941      1 -14.173606   
0.100000    17    -14.323171   0.151192 -14.171978      1 -14.176465   
0.140271    18    -14.325843   0.185646 -14.140196      1 -14.178958   
0.196760    19    -14.327477   0.229092 -14.098386      1 -14.180341   
0.275997    20    -14.326558   0.285280 -14.041278      1 -14.179095   
0.387100    21    -14.319311   0.359583 -13.959728      1 -14.171494   
0.543100    22    -14.292482   0.458595 -13.833887      1 -14.145145   
0.761700    23    -14.238761   0.573463 -13.665298      1 -14.093050   
1.068500    24    -14.170056   0.689104 -13.480952      1 -14.026438   
1.498800    25    -14.092345   0.810720 -13.281626      1 -13.951039   
2.102400    26    -14.004916   0.946815 -13.058101      1 -13.865600   
2.949000    27    -13.908836   1.101970 -12.806866      1 -13.770961   
4.136600    28    -13.814071   1.282194 -12.531876      1 -13.675870   
5.802500    29    -13.730627   1.508870 -12.221757      1 -13.591304   
8.139200    30    -13.651048   1.817300 -11.833748      1 -13.509517   
11.417000   31    -13.570396   2.226787 -11.343610      1 -13.425476   
16.014700   32    -13.503915   2.745508 -10.758406      1 -13.355103   
22.464000   33    -13.505078   3.389882 -10.115196      1 -13.355045   
31.510500   34    -13.601566   4.235268  -9.366298      1 -13.453280   
44.200100   35    -13.746123   5.354838  -8.391286      1 -13.599048   
62.000000   36    -13.916374   6.688273  -7.228101      1 -13.768292   
85.775000   37    -14.071263   8.066562  -6.004701      1 -13.921274   
109.550000  38    -14.143895   9.096176  -5.047719      1 -13.989741   
133.325000  39    -14.172398   9.822188  -4.350210      1 -14.011545   
157.100000  40    -14.205691  10.335774  -3.869917      1 -14.037127   
180.875000  41    -14.247556  10.734246  -3.513310      1 -14.070581   
204.650000  42    -14.300698  11.057142  -3.243556      1 -14.115326   
228.425000  43    -14.369705  11.326057  -3.043648      1 -14.177775   
252.200000  44    -14.464683  11.554138  -2.910545      1 -14.269900   
275.975000  45    -14.627340  11.751017  -2.876323      1 -14.434846   
299.750000  46    -15.017750  11.950421  -3.067329      1 -14.835761   
323.525000  47    -15.688954  12.332850  -3.356104      1 -15.523120   
347.300000  48    -16.469042  13.011291  -3.457751      1 -16.321573   
371.075000  49    -17.249182  13.847138  -3.402045      1 -17.118689   
394.850000  50    -18.019006  14.719883  -3.299123      1 -17.903413   
418.625000  51    -18.776736  15.602191  -3.174545      1 -18.674045   
442.400000  52    -19.520750  16.483354  -3.037396      1 -19.429419   
466.175000  53    -20.249048  17.356632  -2.892416      1 -20.169

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')